# Behavioural Driven Development Testing for Jupyter Notebooks

Handy way to process the run unit tests (via doctest) and integration tests (via behave) in jupyter notebooks (.ipynb) containing Python functions.
The script will convert an .ipynb to a string format (basically a .py file), loads them as modules, and runs the tests on them.
To run it in the console, do:

    python -m pytest --verbose --disable-warnings --nbval test_ipynb.ipynb

The script should tell you which ipynb file's doctests has failed (e.g. srgan_train.ipynb).
You can then open up this very jupyter notebook to debug and inspect the situation further.

In [1]:
from features.environment import _load_ipynb_modules
import behave.__main__

import doctest
import os
import sys


def _unit_test_ipynb(path: str):
    """
    Unit tests on loaded modules from a .ipynb file.
    Uses doctest.
    """
    assert path.endswith(".ipynb")

    module = _load_ipynb_modules(ipynb_path=path)
    num_failures, num_attempted = doctest.testmod(m=module, verbose=True)
    if num_failures > 0:
        sys.exit(num_failures)


def _integration_test_ipynb(path: str, summary: bool = False):
    """
    Integration tests on various feature behaviours inside a .feature file.
    Uses behave.
    """
    assert os.path.exists(path=path)
    assert path.endswith(".feature")

    if summary == False:
        args = f"--tags ~@skip --no-summary {path}"
    elif summary == True:
        args = f"--tags ~@skip {path}"

    num_failures = behave.__main__.main(args=args)
    if num_failures > 0:
        sys.exit(num_failures)

## Unit tests
Uses [doctest](https://en.wikipedia.org/wiki/Doctest).
Small tests for each individual function.

In [2]:
_unit_test_ipynb(path="data_prep.ipynb")

Trying:
    os.makedirs(name="/tmp/highres", exist_ok=True)
Expecting nothing
ok
Trying:
    download_to_path(path="/tmp/highres/2011_Antarctica_TO.csv",
                     url="https://data.cresis.ku.edu/data/rds/2011_Antarctica_TO/csv_good/2011_Antarctica_TO.csv")
Expecting:
    <Response [200]>
ok
Trying:
    _ = shutil.copy(src="highres/20xx_Antarctica_TO.json", dst="/tmp/highres")
Expecting nothing
ok
Trying:
    df = ascii_to_xyz(pipeline_file="/tmp/highres/20xx_Antarctica_TO.json")
Expecting nothing
ok
Trying:
    df.head(2)
Expecting:
                   x             y         z
    0  345580.826265 -1.156471e+06 -377.2340
    1  345593.322948 -1.156460e+06 -376.6332
ok
Trying:
    shutil.rmtree(path="/tmp/highres")
Expecting nothing
ok
Trying:
    download_to_path(path="highres/Data_20171204_02.csv",
                     url="https://data.cresis.ku.edu/data/rds/2017_Antarctica_Basler/csv_good/Data_20171204_02.csv")
Expecting:
    <Response [200]>
ok
Trying:
    check_sha256(

In [3]:
_unit_test_ipynb(path="srgan_train.ipynb")

Using TensorFlow backend.


Trying:
    metrics = {"generator_network": 'mse', "discriminator_network": 'accuracy'}
Expecting nothing
ok
Trying:
    models = compile_srgan_model(
        g_network=generator_network(),
        d_network=discriminator_network(),
        metrics=metrics,
    )
Expecting nothing
ok
Trying:
    models['discriminator_model'].trainable
Expecting:
    True
ok
Trying:
    models['srgan_model'].get_layer(name='generator_network').trainable
Expecting:
    True
ok
Trying:
    models['srgan_model'].get_layer(name='discriminator_network').trainable
Expecting:
    False
ok
Trying:
    models['srgan_model'].count_params()
Expecting:
    8442626
ok
Trying:
    discriminator_network().input_shape
Expecting:
    (None, 32, 32, 1)
ok
Trying:
    discriminator_network().output_shape
Expecting:
    (None, 1)
ok
Trying:
    discriminator_network().count_params()
Expecting:
    6828033
ok
Trying:
    generator_network().input_shape
Expecting:
    [(None, 10, 10, 1), (None, 100, 100, 1), (None, 20, 20, 1

## Integration tests

Uses [behave](https://github.com/behave/behave).
Medium sized tests which checks that components work together properly.
Ensures that the behaviour of features (made up of units) is sound.

In [4]:
_integration_test_ipynb(path="features/data_prep.feature")

@fixture.data_prep
Feature: Data preparation # features/data_prep.feature:3
  In order to have reproducible data inputs for everyone
  As a data scientist,
  We want to share cryptographically secured pieces of the datasets
  Scenario Outline: Download and check data -- @1.1 Files to download and check                                                          # features/data_prep.feature:15
    Given this https://data.cresis.ku.edu/data/rds/2017_Antarctica_Basler/csv_good/Data_20171204_02.csv link to a file hosted on the web # features/steps/test_data_prep.py:8
    When we download it to highres/Data_20171204_02.csv                                                                                  # features/steps/test_data_prep.py:13
    Then the local file should have this 53cef7a0d28ff92b30367514f27e888efbc32b1bda929981b371d2e00d4c671b checksum                       # features/steps/test_data_prep.py:19

  Scenario Outline: Download and check data -- @1.2 Files to download and check   

In [5]:
_integration_test_ipynb(path="features/deepbedmap.feature")

@fixture.deepbedmap
Feature: DeepBedMap # features/deepbedmap.feature:3
  In order to create a great map of Antarctica's bed
  As a scientist,
  We want a model that produces realistic images from many open datasets
  Scenario Outline: Determine high resolution bed -- @1.1 Bounding box views of Antarctica  # features/deepbedmap.feature:16
    Given some view of Antarctica -1593714.328,-164173.7848,-1575464.328,-97923.7848        # features/steps/test_deepbedmap.py:6
    When we gather low and high resolution images related to that view                      # features/steps/test_deepbedmap.py:14
    And pass those images into our trained neural network model                             # features/steps/test_deepbedmap.py:30
    Then a four times upsampled super resolution bed elevation map is returned              # features/steps/test_deepbedmap.py:40

